47 遅延属性には__getattr__, __getattribute__, __setattr__を使う

In [1]:
#クラスが__getattr__を定義しているなら, オブジェクトのインスタンス辞書に属性が見つからないときはいつも__getattr__が呼び出される

class LazyRecord:
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = f'Value for {name}'
        setattr(self, name, value)

In [2]:
data = LazyRecord()
print('Before:', data.__dict__)
print('foo: ',data.foo)
print('After: ', data.__dict__)

Before: {'exists': 5}
foo:  None
After:  {'exists': 5, 'foo': 'Value for foo'}


In [4]:
#__getattribute__属性辞書にそれが存在する場合にも, 属性がオブジェクトでアクセスするたびに呼ばれる

class ValidatingRecord:
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print(f' called __getattribute__({name!r}')
        try:
            value = super().__getattribute__(name)
            print(f'* Found {name!r} returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            print(f'f * Setting {name!r} to {value!r}')
            setattr(self, name, value)
            return value

data = ValidatingRecord()
print('exists: ', data.exists)
print('First foo: ', data.foo)
print('Second foo: ', data.foo)
#__getattr__は要素が存在するときは呼び出されない
#__getattribute__は要素が存在するときも呼び出される

 called __getattribute__('exists'
* Found 'exists' returning 5
exists:  5
 called __getattribute__('foo'
f * Setting 'foo' to 'Value for foo'
First foo:  Value for foo
 called __getattribute__('foo'
* Found 'foo' returning 'Value for foo'
Second foo:  Value for foo


In [5]:
class LoggingLazyRecord(LazyRecord):
    def __getattr__(self, name):
        print(f'Called __getattr__({name!r}),'
                f'populating instance dictionary')
        result = super().__getattr__(name)
        print(f'* Returning {result!r}')
        return result

In [6]:
data = LoggingLazyRecord()
print('exists: ', data.exists)
print('First foo: ', data.foo)
print('Second foo: ', data.foo)


exists:  5
Called __getattr__('foo'),populating instance dictionary
* Returning None
First foo:  None
Second foo:  Value for foo


In [7]:
data = LoggingLazyRecord()
print('Before: ', data.__dict__)
print('Has first foo: ', hasattr(data, 'foo'))
print('Has Second foo: ', hasattr(data,'foo'))

Before:  {'exists': 5}
Called __getattr__('foo'),populating instance dictionary
* Returning None
Has first foo:  True
Has Second foo:  True


In [8]:
data = ValidatingRecord()
print('Has first foo: ', hasattr(data, 'foo'))
print('Has second foo: ', hasattr(data, 'foo'))

 called __getattribute__('foo'
f * Setting 'foo' to 'Value for foo'
Has first foo:  True
 called __getattribute__('foo'
* Found 'foo' returning 'Value for foo'
Has second foo:  True


In [9]:
#__setattr__は属性がインスタンスで代入されるたびに常に呼び出される
class LoggingSavingRecord:
    def __setattr__(self, name, value):
        print(f'* Called __setattr__({name!r}, {value!r})')
        super().__setattr__(name, value)

data = LoggingSavingRecord()
print('Before: ', data.__dict__)
data.foo = 5
print('After: ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)


Before:  {}
* Called __setattr__('foo', 5)
After:  {'foo': 5}
* Called __setattr__('foo', 7)
Finally: {'foo': 7}


In [11]:
class BrokenDictionaryRecord:
    def __init__(self, data):
        self._data = {}

    def __getattribute__(self, name):
        print(f'* called __getattribute__{name!r}')
        return self._data[name]


In [14]:
data = BrokenDictionaryRecord({'foo':3})
data.foo
#__getattribute__がself.__dataにアクセスすると__getattribute__が再度実行され, それがself._dataに再度アクセスするように再帰する

* called __getattribute__'foo'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __getattribute__'_data'
* called __g

RecursionError: maximum recursion depth exceeded while calling a Python object

In [15]:
class DictionaryRecord:
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print(f'* Called __getattribute__({name!r})')
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

In [16]:
data = DictionaryRecord({'foo': 3})
print('foo: ', data.foo)

* Called __getattribute__('foo')
foo:  3


項目48 サブクラスを__init_subclss__で検証する

In [17]:
#クラスの検証コードはクラスの方を持つオブジェクトが作られるときの__init__メソッドの中で実行されることが多い

In [18]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print(f'* Running {meta}.__new__ for {name}')
        print('Based: ', bases)
        print(class_dict)
        return type.__new__(meta, name, bases, class_dict)

In [20]:
class MyClass(metaclass = Meta):
    stuff = 123

    def foo(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MyClass
Based:  ()
{'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x7f735820c950>}


In [21]:
class MySubclass(MyClass):
    other = 567

    def bar(self):
        pass

* Running <class '__main__.Meta'>.__new__ for MySubclass
Based:  (<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'other': 567, 'bar': <function MySubclass.bar at 0x7f7357f0c710>}


In [22]:
#メタクラスはクラス名、それが継承しているスーパークラス、class本体で定義されている全てのクラス属性にアクセスできる. 全てのクラスはobjectを
#継承するので, 基底クラスのタプルにはobjectを明示的に含む必要はない

In [30]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        if bases:
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')

        return type.__new__(meta, name, bases, class_dict)

In [31]:
class Polygon(object, metaclass= ValidatePolygon):
    sides = None

    @classmethod
    def interior_angles(cls):
        return (cls.sides-2) * 180

class Triangle(Polygon):
    sides = 3

class Rectangle(Polygon):
    sides = 4

class Nonagon(Polygon):
    sides = 9

assert Triangle.interior_angles()==180
assert Rectangle.interior_angles() ==360
assert Nonagon.interior_angles()==1260

TypeError: '<' not supported between instances of 'NoneType' and 'int'